# Finding the frauds

### This Ipython notebook aims to find out all the likely fraudsters based on certain features and user behavior.

In [2]:
import pandas as pd
import os
from file import *

### Loading the data from csv files to dataframes using pandas

In [3]:
currency_df = pd.read_csv(os.path.join(dir_path, './fct_data/currency_details.csv'))
fx_df = pd.read_csv(os.path.join(dir_path, './fct_data/fx_rates.csv'))
trans_df = pd.read_csv(os.path.join(dir_path, './fct_data/transactions.csv'), index_col=0)
users_df = pd.read_csv(os.path.join(dir_path, './fct_data/users.csv'), index_col=0)
fraudster_df = pd.read_csv(os.path.join(dir_path, './fct_data/fraudsters.csv'), index_col=0)


Converting the column headers to lowercase

In [4]:
currency_df.columns = map(str.lower, currency_df.columns)
fx_df.columns = map(str.lower, fx_df.columns)
trans_df.columns = map(str.lower, trans_df.columns)
users_df.columns = map(str.lower, users_df.columns)

# Explore the data
trans_df.head()

,currency,amount,state,created_date,merchant_category,merchant_country,entry_method,user_id,type,source,id
0,GBP,3738,COMPLETED,2015-10-11 09:05:43.016000,bar,AUS,misc,7285c1ec-31d0-4022-b311-0ad9227ef7f4,CARD_PAYMENT,GAIA,5a9ee109-e9b3-4598-8dd7-587591e6a470
1,GBP,588,COMPLETED,2015-10-11 20:08:39.150000,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,28d68bf4-460b-4c8e-9b95-bcda9ab596b5
2,GBP,1264,COMPLETED,2015-10-11 11:37:40.908000,NaN,UKR,misc,0fe472c9-cf3e-4e43-90f3-a0cfb6a4f1f0,CARD_PAYMENT,GAIA,1f1e8817-d40b-4c09-b718-cfc4a6f211df
3,GBP,66,REVERTED,2015-10-11 20:08:35.310000,NaN,CA,misc,20100a1d-12bc-41ed-a5e1-bc46216e9696,CARD_PAYMENT,GAIA,a7aaf78c-d201-456f-9e6d-612a795e8c32
4,GBP,968,COMPLETED,2015-10-11 02:46:47.640000,supermarket,NZL,misc,821014c5-af06-40ff-91f4-77fe7667809f,CARD_PAYMENT,GAIA,27dd99a2-5539-4ba9-876a-1a94abc2701f


### Approach to find the fraudsters

Looking at the data, you must first list down all the potential features which are crucial in the process of a transaction. Listing down all the important factors which can provide some leads:
1. KYC - Legal Identification status of the user
2. has_email - if the user has a registered email in our database
3. state - Is he active or passive or locked?
4. Failed sign in attempts - In case of a hacked account
5. Merchant country - Look at the country to which the amount is being transferred

Now, we need to build a potential scenario by knitting together all the above parameters.
We are basically testing the user's authenticity from all the parameters mentioned above.

Logic behind declaring someone as a Fraudster:
- If a user has its KYC failed with no email address registered and is actively using our platform.
- Above this, check if the country is recorded in the transaction to which the transfer has been made.

In [5]:
# finding out all the unidentified users who have their KYC failed
unidentified_df = users_df.query('kyc == "FAILED" and has_email == 0 and state == "ACTIVE"')

# merging(joining) the above dataframe with the transactional database on user_id
unidentified_txns = pd.merge(unidentified_df, trans_df, left_on='id', right_on='user_id')

unidentified_txns

,failed_sign_in_attempts,kyc,birth_year,country,state_x,created_date_x,terms_version,phone_country,has_email,id_x,...,amount,state_y,created_date_y,merchant_category,merchant_country,entry_method,user_id,type,source,id_y
0,0,FAILED,1966,GB,ACTIVE,2015-11-06 12:39:34.355000,NaN,GB||JE||IM||GG,0,62edb2a5-8b28-466a-9a9a-b3e314f2f771,...,1000,DECLINED,2015-11-17 20:30:33.796000,NaN,NaN,misc,62edb2a5-8b28-466a-9a9a-b3e314f2f771,ATM,GAIA,9d99f7f9-ba47-42e4-8151-a5a2aef27f96
1,0,FAILED,1993,GE,ACTIVE,2015-10-18 07:02:26.487000,NaN,GE,0,f6b7f148-bb9f-42c9-9c56-9243f83ee3d5,...,1000,FAILED,2015-11-10 20:26:34.738000,NaN,NaN,misc,f6b7f148-bb9f-42c9-9c56-9243f83ee3d5,TOPUP,NYX,d659b109-0509-49b1-af70-b9a2158b33a4
2,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,2585,COMPLETED,2015-09-24 11:31:29.321000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,e60174ba-59d1-4084-bcbe-43152f7d22e4
3,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,3320,COMPLETED,2015-09-27 08:25:57.976000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,74d947bd-49e9-42f4-ba74-e18d86fa29d9
4,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,5535,COMPLETED,2015-09-27 14:18:17.138000,point_of_interest,GBR,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,65c7a581-6cbd-4202-9666-f67791ae6c7d
5,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,10624,COMPLETED,2015-09-26 12:02:31.562000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,29fca723-8a58-4660-bc54-465d553436ca
6,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,1362,COMPLETED,2015-09-25 16:21:11.801000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,2a05136d-1bf8-469c-85c5-53cbbda5af21
7,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,3537,COMPLETED,2015-09-25 17:56:50.974000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,f4a1c88d-0f8d-4e43-bcef-4b6a053165a6
8,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,2066,COMPLETED,2015-09-26 12:12:18.467000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,1fcd497c-b702-442c-a062-4598991b163f
9,0,FAILED,1981,GB,ACTIVE,2015-08-09 07:33:29.491000,NaN,GB||JE||IM||GG,0,82a2ecd2-0b94-4629-85c0-141d7af04e5f,...,9591,COMPLETED,2015-09-26 06:03:14.521000,NaN,ITA,misc,82a2ecd2-0b94-4629-85c0-141d7af04e5f,CARD_PAYMENT,GAIA,e6bf4bbd-ee06-4644-b389-19d733f7faea


Among the unidentified users, we will check for their transactional data to check for the merchant country. If the country to which the transfer has been made is captured. Narrowing down the list of users who are most likely to be frauds.

In [6]:
# checking for NaN values in the merchant_country column in the above obtained dataframe
frauds_df = unidentified_txns[unidentified_txns['merchant_country'].isnull()]

In [7]:
trans_df['created_date'] = trans_df['created_date'].astype('datetime64[s]').dt.date
freq_trans_df = trans_df.groupby(['created_date', 'user_id']).count()
a = freq_trans_df[freq_trans_df['amount'] > 50].reset_index(level=0, drop=True)
freq_txn_user_ids = a.index.unique()
freq_txn_user_ids

Index(['ec4fd825-7167-450b-aa9f-0cbcf681978b',
       '1e3ad24b-8818-4db9-82fb-a0c30fc5b35a',
       '073abac2-836b-4964-9005-d0aed492cd96',
       'abfc3bdc-f09c-4eff-982c-fb8af72b07ad',
       'a24b204c-3b6a-446a-8b06-20dd150a4805'],
      dtype='object', name='user_id')

### Removing all the already obtained fraudsters in the list

In [8]:
fraud_id_set = set(frauds_df['id_x']).union(set(freq_txn_user_ids))
potential_fraudsters = fraud_id_set - set(fraudster_df['user_id'])
potential_fraudsters

{'073abac2-836b-4964-9005-d0aed492cd96',
 '146cb748-d9a7-410a-b2ec-f5c403f7149d',
 '16102a27-534e-48d7-b7d2-99e08e6c7597',
 '1e3ad24b-8818-4db9-82fb-a0c30fc5b35a',
 '1eb7c725-f748-4e76-98cc-1bb6b58d4e4a',
 '5302559e-9dbd-4f1d-ba3b-4e5a10f0e51a',
 '5ff27f1c-65e2-426f-a139-56d7cf6f4a42',
 '62edb2a5-8b28-466a-9a9a-b3e314f2f771',
 '78a0cfa2-f9d3-4f07-94c4-a97f76595589',
 '82a2ecd2-0b94-4629-85c0-141d7af04e5f',
 'a24b204c-3b6a-446a-8b06-20dd150a4805',
 'abfc3bdc-f09c-4eff-982c-fb8af72b07ad',
 'acbb5850-8df2-4614-a277-83e020a9edc9',
 'c876e9f6-c0f8-4ee6-80a0-2a9ef751cc80',
 'ccc08947-8246-45c3-ae88-8f9327b60a74',
 'd1e9aea4-e0e3-4ac3-af74-f8095c8b8861',
 'd4aee110-f821-4545-b617-1f49eacad2b9',
 'dd8e1dd7-e467-41ff-908f-4e0ab64d9af3',
 'e0ff10b4-4e05-45c8-9183-ab45ae37913e',
 'e14f2def-d412-4ba6-90da-6ff8c8bf2782',
 'ec4fd825-7167-450b-aa9f-0cbcf681978b',
 'f6b7f148-bb9f-42c9-9c56-9243f83ee3d5'}

In [9]:
# Here is the final list of potential fraudsters on our platform.
potential_fraudsters

{'073abac2-836b-4964-9005-d0aed492cd96',
 '146cb748-d9a7-410a-b2ec-f5c403f7149d',
 '16102a27-534e-48d7-b7d2-99e08e6c7597',
 '1e3ad24b-8818-4db9-82fb-a0c30fc5b35a',
 '1eb7c725-f748-4e76-98cc-1bb6b58d4e4a',
 '5302559e-9dbd-4f1d-ba3b-4e5a10f0e51a',
 '5ff27f1c-65e2-426f-a139-56d7cf6f4a42',
 '62edb2a5-8b28-466a-9a9a-b3e314f2f771',
 '78a0cfa2-f9d3-4f07-94c4-a97f76595589',
 '82a2ecd2-0b94-4629-85c0-141d7af04e5f',
 'a24b204c-3b6a-446a-8b06-20dd150a4805',
 'abfc3bdc-f09c-4eff-982c-fb8af72b07ad',
 'acbb5850-8df2-4614-a277-83e020a9edc9',
 'c876e9f6-c0f8-4ee6-80a0-2a9ef751cc80',
 'ccc08947-8246-45c3-ae88-8f9327b60a74',
 'd1e9aea4-e0e3-4ac3-af74-f8095c8b8861',
 'd4aee110-f821-4545-b617-1f49eacad2b9',
 'dd8e1dd7-e467-41ff-908f-4e0ab64d9af3',
 'e0ff10b4-4e05-45c8-9183-ab45ae37913e',
 'e14f2def-d412-4ba6-90da-6ff8c8bf2782',
 'ec4fd825-7167-450b-aa9f-0cbcf681978b',
 'f6b7f148-bb9f-42c9-9c56-9243f83ee3d5'}

### Now, among these likely fraudsters, we'll shortlist a few high potential fraudsters who are making transfers of amount more than 20000.

In [10]:
high_pot_frauds_df = frauds_df[frauds_df['amount'] > 2000]

In [11]:
high_pot_fraudsters = set(high_pot_frauds_df['id_x']) - set(fraudster_df['user_id']) - set(freq_txn_user_ids)

### Ids of the high risk users.

In [12]:
high_pot_fraudsters

{'146cb748-d9a7-410a-b2ec-f5c403f7149d',
 '16102a27-534e-48d7-b7d2-99e08e6c7597',
 '1eb7c725-f748-4e76-98cc-1bb6b58d4e4a',
 '5302559e-9dbd-4f1d-ba3b-4e5a10f0e51a',
 '5ff27f1c-65e2-426f-a139-56d7cf6f4a42',
 '82a2ecd2-0b94-4629-85c0-141d7af04e5f',
 'acbb5850-8df2-4614-a277-83e020a9edc9',
 'c876e9f6-c0f8-4ee6-80a0-2a9ef751cc80',
 'dd8e1dd7-e467-41ff-908f-4e0ab64d9af3'}

## Financial crime Typology and Due Diligence

We have clearly focused on the legal identification of the user to be able to transact the money.

Users who don't have their KYC done, we don't have any record or method to:
- source of income
- assess further potential risks like money laundering and other internet sourced frauds.
- make sure that all our clients are anti-bribery compliant
- to put in place an anti-corruption due diligence 

Keeping in mind the above mentioned crucial factors, here is how we can conduct due diligence on these high risk users:
1. Revoking their permission to make any transaction before they get their KYC PASSED.
2. For an amount greater than 20000, Merchant's country should be made mandatory to be supplied.
3. Email should be verified along with the phone number.
4. All the tax reciepts / PAN(Anti-corruption information) card details should be verified. 


# ============================================================

## Question 3(a)

Exploring the already figured fraudsters in the fraudsters.csv. Mapping the relationships between fraudsters, users and transactions files.

Steps:
1. Pick up an id from the fraudsters file and run it over in the users and transactions file.
2. Look for a pattern which is common or an anomaly in all the columns in the two files.
3. To make it easy, we'll merge the fraudsters dataframe with the users df and another datafram with the transactions dataframe.


In [73]:
# Merging the fraudster dataframe with the transaction to understand the transaction patterns of fraudsters.

fr_txn_df = pd.merge(fraudster_df, trans_df, on="user_id", how = "left")

"""
get the ids of the users who have:
1. transacted more than 500000 in amount,
2. Don't have the merchant country specified
""" 
ids = trans_df.query('amount > 100000 and merchant_country.isnull()')['user_id'].unique()
len(set(ids).intersection(set(fraudster_df['user_id'])))

146

### Merging the fraudster dataframe with the users dataframe to understand the user information of fraudsters.

In [ ]:
fr_user_df = pd.merge(fraudster_df, users_df, left_on=["user_id"], right_on = ["id"],  how ="left")

In [15]:
fr_user_df

,user_id,failed_sign_in_attempts,kyc,birth_year,country,state,created_date,terms_version,phone_country,has_email,id
0,5270b0f4-2e4a-4ec9-8648-2135312ac1c4,0,NONE,1988,GB,LOCKED,2017-09-19 01:55:22.376000,NaN,GB||JE||IM||GG,1,5270b0f4-2e4a-4ec9-8648-2135312ac1c4
1,848fc1b1-096c-40f7-b04a-1399c469e421,0,PASSED,1994,GB,LOCKED,2018-06-16 15:55:43.800000,2018-05-25,PL,1,848fc1b1-096c-40f7-b04a-1399c469e421
2,27c76eda-e159-4df3-845a-e13f4e28a8b5,0,PASSED,1997,GB,LOCKED,2018-05-11 18:20:45.608000,2018-05-25,GB||JE||IM||GG,1,27c76eda-e159-4df3-845a-e13f4e28a8b5
3,a27088ef-9452-403d-9bbb-f7b10180cdda,0,PASSED,1989,BE,LOCKED,2018-05-18 16:41:12.267000,2018-05-25,IN,1,a27088ef-9452-403d-9bbb-f7b10180cdda
4,fb23710b-609a-49bf-8a9a-be49c59ce6de,0,PASSED,1992,LT,LOCKED,2018-03-29 13:37:29.736000,2018-03-20,LT,1,fb23710b-609a-49bf-8a9a-be49c59ce6de
5,3a1e97cd-69a7-4c68-a98c-e0d7d606b07b,0,PASSED,1989,GB,LOCKED,2018-04-27 19:12:48.073000,2018-05-25,GB||JE||IM||GG,1,3a1e97cd-69a7-4c68-a98c-e0d7d606b07b
6,e7b6e022-76ca-4fe9-992e-0773a4715b0c,0,PENDING,1997,GB,LOCKED,2017-08-16 15:14:31.950000,2018-03-20,GB||JE||IM||GG,1,e7b6e022-76ca-4fe9-992e-0773a4715b0c
7,e7876b06-bcd8-4193-9eaa-b477313f6f1a,0,PASSED,1978,LT,LOCKED,2018-06-11 14:58:25.637000,2018-05-25,LT,1,e7876b06-bcd8-4193-9eaa-b477313f6f1a
8,5b6f20a3-b39a-4f1b-bc5a-974daab4d2ec,0,FAILED,1989,GB,LOCKED,2018-02-24 15:56:08.903000,2018-03-20,GB||JE||IM||GG,1,5b6f20a3-b39a-4f1b-bc5a-974daab4d2ec
9,7377f2fc-4585-49e8-8896-df332efc51c8,0,PASSED,1998,GB,LOCKED,2016-04-07 12:45:30.913000,NaN,GB||JE||IM||GG,1,7377f2fc-4585-49e8-8896-df332efc51c8


### Looking at the number of transactions done by a user in a day

In [77]:
# Grouping by created_date and user_id in the transaction dataframes.

trans_df['created_date'] = trans_df['created_date'].astype('datetime64[s]').dt.date
freq_frd_txn_df = trans_df.groupby(['created_date', 'user_id']).count()

# for over 10 transactions in a day by a normal user
freq_frd_txn_df = freq_frd_txn_df[freq_frd_txn_df['amount'] > 5].reset_index(level=0, drop=True)
usr_ids = freq_frd_txn_df.index.unique()
set(usr_ids).intersection(set(fraudster_df['user_id']))

{'0180632d-7737-42af-aaf0-95c2714d7854',
 '018f9e74-458b-4782-8a92-5f95734ddcd7',
 '024320ea-0cfa-4ce9-8d44-ad9fbfaf29e4',
 '03025e4f-45c8-4755-bece-be36e98b73af',
 '06e026bb-658c-4691-bbb0-7795148403e2',
 '0782ced4-7bb6-4420-aff2-ed08290ee47c',
 '09f9f5ce-e041-49e0-a9f1-7f731bc5fd2b',
 '0bed6203-0ed0-4e40-8f9c-e52ed7dedb6a',
 '0dcc7887-c1ee-4344-94e1-a67ae5d2d11f',
 '0f68c527-fc39-4a9a-8eba-7f46f6dfd593',
 '1193656b-6e8f-4f0c-9824-5c66c588f219',
 '11f8f705-c675-4b59-b06a-e0256c3a6be6',
 '11fa06ae-839a-4279-b85e-191f57d0138a',
 '1206fe8d-e1fb-4965-acf5-07b9e3b84462',
 '12c1cfcc-2baa-4e6f-b6fd-198a30d1d3d6',
 '146dca23-5b81-449f-98b7-4809f52c3959',
 '15ba079f-d680-4ed9-acfb-10e95594e3d6',
 '1632f5e0-e2da-4668-b487-5cc897b963d3',
 '16aee2d9-7d04-4599-8fa3-52146ab10f72',
 '187699b8-aa4d-48d4-bd8b-9e450943c589',
 '1ac43c83-6c00-41cb-943c-7279a79ff2aa',
 '1c79de57-df26-407a-b7ab-160a774b8486',
 '1d3828f5-2e5c-47f3-b304-4e695d1603e3',
 '1d4a5b03-fd14-4bda-bf34-275600f73248',
 '1dca7243-ef3d-